In [2]:
#!/usr/bin/python2
import numpy as np
from numpy.lib.stride_tricks import as_strided

import chainer
from chainer import computational_graph
from chainer import cuda
from chainer import optimizers
from chainer import serializers

import argparse
import six
import imageio
import numbers
     
from nr_model import Model
from fr_model import FRModel

RuntimeError: CUDA environment is not correctly set up
(see https://github.com/chainer/chainer#installation).No module named cupy